Uploader le fichier products.csv. Repérer les valeurs aberrantes dans le fichier (ordre de grandeur : plusieurs centaines).

In [0]:
%fs ls /FileStore/tables

In [0]:
product_df = spark.read.csv("/FileStore/tables/SPN_products.csv", header=True, inferSchema=True)

In [0]:
from pyspark.sql.functions import col

columns_to_sum = [
    "fat_100g",
    "sugars_100g",
    "fiber_100g", 
    "proteins_100g",
    "salt_100g",
    "autre",
]

condition = sum([col(column) for column in columns_to_sum]) > 100
df_sum_more_than_100 = product_df.filter(condition)

In [0]:
display(df_sum_more_than_100.show())
print(df_sum_more_than_100.count())

+----+-------------+--------+------------------+-----------+----------+-------------+---------+-----------+-----+
| _c0|         code|fat_100g|saturated-fat_100g|sugars_100g|fiber_100g|proteins_100g|salt_100g|sodium_100g|autre|
+----+-------------+--------+------------------+-----------+----------+-------------+---------+-----------+-----+
|  87|      34449.0|   57.14|              7.14|       3.57|      17.9|        21.43|  1.08966|      0.429|  0.0|
| 310| 6.09231508E8|   33.33|             21.43|      57.14|       0.0|         9.52|  0.18034|      0.071|  0.0|
| 318| 6.09941759E8|   33.33|             19.44|      55.56|       2.8|         8.33|  0.21082|      0.083|  0.0|
| 319| 6.09941766E8|   33.33|             19.44|      55.56|       2.8|         8.33|  0.21082|      0.083|  0.0|
| 324| 6.09983001E8|   33.33|             19.44|      55.56|       2.8|         8.33|  0.21082|      0.083|  0.0|
| 646| 6.44600352E9|    40.0|              25.0|       50.0|       2.5|          7.5|   

In [0]:
from pyspark.sql.functions import col
duplicates = product_df.groupBy("code").count().filter(col("count") > 1)
duplicates.count()

Out[14]: 176

In [0]:
from pyspark.sql.functions import col

columns_to_check = [
    "fat_100g",
    "saturated-fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g", 
    "sodium_100g",
    "autre",
]

condition = None
for column in columns_to_check:
    if condition is None:
        condition = col(column) > 100
    else:
        condition |= col(column) > 100

df_sup_100 = product_df.filter(condition)

In [0]:
df_sup_100.show()
print(df_sup_100.count())

+-----+---------------+--------+------------------+-----------+----------+-------------+---------+-----------+-----+
|  _c0|           code|fat_100g|saturated-fat_100g|sugars_100g|fiber_100g|proteins_100g|salt_100g|sodium_100g|autre|
+-----+---------------+--------+------------------+-----------+----------+-------------+---------+-----------+-----+
|  501|  2.124810206E9|     0.0|              null|       null|      null|          0.0|105.83418|     41.667|  0.0|
| 1435| 1.030033529E10|   45.71|              8.57|       5.71|       5.7|        17.14|870.85678|    342.857|  0.0|
| 3472|1.1110825261E10|     0.0|               0.0|       null|      null|          0.0|    101.6|       40.0|  0.0|
| 7037|1.1161036074E10|     0.0|              null|       null|      null|          0.0|    101.6|       40.0|  0.0|
| 7983|1.1213053264E10|    0.77|               0.0|       2.31|       3.8|         4.62|781.53768|    307.692|  0.0|
|11129|1.3964151954E10|   30.65|              2.42|      22.58| 

In [0]:
df_fat_less_than_saturated = product_df.filter(col("fat_100g") < col("saturated-fat_100g"))
df_fat_less_than_saturated.count()

Out[19]: 354